In [1]:
import json
import numpy as np
import os

In [2]:
import progressbar
from bert_serving.client import BertClient
bc = BertClient(ip='10.15.82.136', check_version=False)

In [3]:
root_folder = '/home/lyt/code/concept_vqa'
data_folder = '{}/dataTVQA'.format(root_folder)
ocr_folder = '{}/image-feature/ocr'.format(data_folder)
emb_size = 768
ocr_num = 50

In [4]:
split = 'TVA' # train on Tr; test on Val
# split = 'TVEA' # train on Tr+Val; test on Te

# suffix = '' # ocr_bert_DR
# suffix = '.0init' # ocr_bert_DR.0init
# suffix = '.clean.0init' # ocr_bert_DR.clean.0init
# suffix = '.assemble.0init'
#=======================================
zeroInit = True # True for 0 init; False for random init
if zeroInit:
    suffix = '.0init'
else:
    suffix = ''

In [5]:
jsdict = json.load(open('{}/data.json'.format(data_folder),'r'))
print(jsdict.keys())
itoa = jsdict['itoa']
removeIdx = -1
for i,a in enumerate(itoa):
    if a == '':
        print('index:({}), content:({})'.format(i,a))
        itoa[i] = '<blank>'
        removeIdx = i

dict_keys(['itoa', 'itow'])
index:(20), content:()


In [6]:
if zeroInit:
    ans_emb = np.zeros((len(itoa), emb_size)).astype('float32')
else:
    ans_emb = np.random.randn(len(itoa), emb_size).astype('float32')
print(ans_emb.shape)
ans_emb[0:len(itoa)] = bc.encode(itoa)

(8205, 768)


In [7]:
if removeIdx >= 0:
    if zeroInit:
        ans_emb[removeIdx] = np.zeros(emb_size).astype('float32')
    else:
        ans_emb[removeIdx] = np.random.randn(emb_size).astype('float32')
print(ans_emb.shape)

(8205, 768)


In [8]:
save_folder = '{}/bert.{}{}/{}'.format(ocr_folder, emb_size, suffix, split)
save_path = '{}/ans_emb.npy'.format(save_folder)
print('[Save] save path: {}'.format(save_path))
if not os.path.isdir(save_folder):
    os.makedirs(save_folder)
np.save(save_path, ans_emb)

[Save] save path: /home/lyt/code/concept_vqa/dataTVQA/image-feature/ocr/bert.768.0init/TVA/ans_emb.npy


In [9]:
bc.close()